In [14]:
import sys
sys.path.append('/home/mila/l/leo.gagnon/latent_control')

%load_ext autoreload
%autoreload 2
import torch
import pyvene as pv
import pprint
from task import MetaLearningTask, FineTuningTask
from transformers import BatchEncoding, PretrainedConfig
import jax.random as jr
from torch2jax import j2t, t2j
import dataclasses
import pyreft
import nvidia_smi
from models.mamba import MambaLMHeadModel
from dataclasses import dataclass
from mamba_ssm.utils.generation import InferenceParams
from task import DynamicIntervenableModel, TuneConfig, LoraConfig, ReftConfig
import gc

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
task = FineTuningTask(TuneConfig(
        'b9h7hygi', ReftConfig(low_rank_dimension=2, layers=[1,3,5], timesteps=[-1], component='output'), [[1, 0], [2, 1]]
    ))
task.model.to('cuda');

number of parameters: 18.94M
Loaded dataset : (11288/1000)
Loaded checkpoing : epoch=438,step=19750.ckpt


In [4]:
seqs = task.full_data.__getitems__(torch.arange(64), length=100)[0]

In [28]:
del task
torch.cuda.empty_cache()

In [12]:
logits = task.model(seqs)

In [8]:
logits.shape

torch.Size([64, 100, 51])

In [17]:
import nvidia_smi

nvidia_smi.nvmlInit()

handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
# card id 0 hardcoded here, there is also a call to get all available card ids, so we could iterate

info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)

print("Total memory:", info.total)
print("Free memory:", info.free)
print("Used memory:", info.used)

nvidia_smi.nvmlShutdown()


Total memory: 34359738368
Free memory: 33547485184
Used memory: 812253184


In [16]:
del logits
gc.collect()
torch.cuda.empty_cache()

In [18]:
torch.cuda.empty_cache()